# 5.- Test Local Training

In [ ]:
dockerfile_path = "../docker"
image_tag = "satellite-object-detection:latest"

In [ ]:
# Build docker image
!docker build -f {dockerfile_path}/Dockerfile -t {image_tag} {dockerfile_path}

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.38kB 0.0s done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/ubuntu:20.04
#2 DONE 1.2s

#3 [internal] load .dockerignore
#3 transferring context: 2B 0.0s done
#3 DONE 0.5s

#4 [ 1/18] FROM docker.io/library/ubuntu:20.04@sha256:8e5c4f0285ecbb4ead070431d29b576a530d3166df73ec44affc1cd27555141b
#4 DONE 0.0s

#5 [ 9/18] WORKDIR /models/research
#5 CACHED

#6 [ 3/18] RUN ln -s /usr/bin/python3 /usr/bin/python
#6 CACHED

#7 [ 4/18] RUN python3 -m pip install --no-cache-dir "tensorflow==2.13.0"
#7 CACHED

#8 [ 6/18] RUN git clone --branch master     https://github.com/tensorflow/models.git &&     cd /models &&     git checkout 4a56eff0ad5ce23f64f7638167c3256b36da3b07
#8 CACHED

#9 [ 7/18] WORKDIR /tmp
#9 CACHED

#10 [ 2/18] RUN apt-get update &&     apt-get install -y     curl     git     unzip     protobuf-compiler     build-essenti

In [6]:
!wsl --shutdown

In [ ]:
# Use this if the latest command dees not work
!wsl --unregister docker-desktop